# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,May 26 2022 12:42PM,238168,10,Stone8066556,Acute Outpatient Solutions,Released
1,May 26 2022 12:35PM,238167,20,ALU0006934,Alumier Labs Inc.,Released
2,May 26 2022 12:35PM,238167,20,ALUR486860133,Alumier Labs Inc.,Released
3,May 26 2022 12:35PM,238167,20,ALUR083356035,Alumier Labs Inc.,Released
4,May 26 2022 12:35PM,238167,20,ALUR086724636,Alumier Labs Inc.,Released
5,May 26 2022 12:35PM,238167,20,ALUR203557815,Alumier Labs Inc.,Released
6,May 26 2022 12:35PM,238167,20,ALUR223474165,Alumier Labs Inc.,Released
7,May 26 2022 12:35PM,238167,20,ALUR368801049,Alumier Labs Inc.,Released
8,May 26 2022 12:35PM,238167,20,ALUR408021600,Alumier Labs Inc.,Released
9,May 26 2022 12:35PM,238167,20,ALUR535792034,Alumier Labs Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
33,238164,Released,1
34,238165,Released,9
35,238166,Released,1
36,238167,Released,19
37,238168,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
238164,NaN,NaN,1.0
238165,NaN,NaN,9.0
238166,NaN,NaN,1.0
238167,NaN,NaN,19.0
238168,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238087,0.0,0.0,1.0
238090,5.0,17.0,7.0
238100,0.0,0.0,1.0
238101,0.0,0.0,20.0
238105,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238087,0,0,1
238090,5,17,7
238100,0,0,1
238101,0,0,20
238105,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238087,0,0,1
1,238090,5,17,7
2,238100,0,0,1
3,238101,0,0,20
4,238105,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238087,,,1
1,238090,5,17,7
2,238100,,,1
3,238101,,,20
4,238105,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,May 26 2022 12:42PM,238168,10,Acute Outpatient Solutions
1,May 26 2022 12:35PM,238167,20,Alumier Labs Inc.
20,May 26 2022 12:27PM,238165,20,"ACI Healthcare USA, Inc."
29,May 26 2022 12:22PM,238166,19,ACG North America LLC
30,May 26 2022 12:20PM,238164,10,Bio-PRF
31,May 26 2022 12:20PM,238162,10,Bio-PRF
38,May 26 2022 12:19PM,238163,21,"NBTY Global, Inc."
39,May 26 2022 12:01PM,238161,10,"CLINUVEL, Inc."
41,May 26 2022 11:37AM,238159,18,Hush Hush
42,May 26 2022 11:01AM,238157,10,Eywa Pharma Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,May 26 2022 12:42PM,238168,10,Acute Outpatient Solutions,,,1
1,May 26 2022 12:35PM,238167,20,Alumier Labs Inc.,,,19
2,May 26 2022 12:27PM,238165,20,"ACI Healthcare USA, Inc.",,,9
3,May 26 2022 12:22PM,238166,19,ACG North America LLC,,,1
4,May 26 2022 12:20PM,238164,10,Bio-PRF,,,1
5,May 26 2022 12:20PM,238162,10,Bio-PRF,,,7
6,May 26 2022 12:19PM,238163,21,"NBTY Global, Inc.",,,1
7,May 26 2022 12:01PM,238161,10,"CLINUVEL, Inc.",,,2
8,May 26 2022 11:37AM,238159,18,Hush Hush,,,1
9,May 26 2022 11:01AM,238157,10,Eywa Pharma Inc.,,9,10


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 26 2022 12:42PM,238168,10,Acute Outpatient Solutions,1,,
1,May 26 2022 12:35PM,238167,20,Alumier Labs Inc.,19,,
2,May 26 2022 12:27PM,238165,20,"ACI Healthcare USA, Inc.",9,,
3,May 26 2022 12:22PM,238166,19,ACG North America LLC,1,,
4,May 26 2022 12:20PM,238164,10,Bio-PRF,1,,
5,May 26 2022 12:20PM,238162,10,Bio-PRF,7,,
6,May 26 2022 12:19PM,238163,21,"NBTY Global, Inc.",1,,
7,May 26 2022 12:01PM,238161,10,"CLINUVEL, Inc.",2,,
8,May 26 2022 11:37AM,238159,18,Hush Hush,1,,
9,May 26 2022 11:01AM,238157,10,Eywa Pharma Inc.,10,9,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 26 2022 12:42PM,238168,10,Acute Outpatient Solutions,1,,
1,May 26 2022 12:35PM,238167,20,Alumier Labs Inc.,19,,
2,May 26 2022 12:27PM,238165,20,"ACI Healthcare USA, Inc.",9,,
3,May 26 2022 12:22PM,238166,19,ACG North America LLC,1,,
4,May 26 2022 12:20PM,238164,10,Bio-PRF,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 26 2022 12:42PM,238168,10,Acute Outpatient Solutions,1.0,NaN,NaN
1,May 26 2022 12:35PM,238167,20,Alumier Labs Inc.,19.0,NaN,NaN
2,May 26 2022 12:27PM,238165,20,"ACI Healthcare USA, Inc.",9.0,NaN,NaN
3,May 26 2022 12:22PM,238166,19,ACG North America LLC,1.0,NaN,NaN
4,May 26 2022 12:20PM,238164,10,Bio-PRF,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 26 2022 12:42PM,238168,10,Acute Outpatient Solutions,1.0,0.0,0.0
1,May 26 2022 12:35PM,238167,20,Alumier Labs Inc.,19.0,0.0,0.0
2,May 26 2022 12:27PM,238165,20,"ACI Healthcare USA, Inc.",9.0,0.0,0.0
3,May 26 2022 12:22PM,238166,19,ACG North America LLC,1.0,0.0,0.0
4,May 26 2022 12:20PM,238164,10,Bio-PRF,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1905221,44.0,22.0,15.0
15,476280,5.0,19.0,0.0
16,714351,2.0,1.0,0.0
18,476292,4.0,8.0,0.0
19,714361,3.0,0.0,0.0
20,1666969,73.0,23.0,5.0
21,476284,2.0,0.0,0.0
22,476226,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1905221,44.0,22.0,15.0
1,15,476280,5.0,19.0,0.0
2,16,714351,2.0,1.0,0.0
3,18,476292,4.0,8.0,0.0
4,19,714361,3.0,0.0,0.0
5,20,1666969,73.0,23.0,5.0
6,21,476284,2.0,0.0,0.0
7,22,476226,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,44.0,22.0,15.0
1,15,5.0,19.0,0.0
2,16,2.0,1.0,0.0
3,18,4.0,8.0,0.0
4,19,3.0,0.0,0.0
5,20,73.0,23.0,5.0
6,21,2.0,0.0,0.0
7,22,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,44.0
1,15,Released,5.0
2,16,Released,2.0
3,18,Released,4.0
4,19,Released,3.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,18,19,20,21,22
Status,,,,,,,,
Completed,15.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0
Executing,22.0,19.0,1.0,8.0,0.0,23.0,0.0,0.0
Released,44.0,5.0,2.0,4.0,3.0,73.0,2.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,18,19,20,21,22
0,Completed,15.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0
1,Executing,22.0,19.0,1.0,8.0,0.0,23.0,0.0,0.0
2,Released,44.0,5.0,2.0,4.0,3.0,73.0,2.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,18,19,20,21,22
0,Completed,15.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0
1,Executing,22.0,19.0,1.0,8.0,0.0,23.0,0.0,0.0
2,Released,44.0,5.0,2.0,4.0,3.0,73.0,2.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()